In [1]:
import tensorflow as tf
old_v = tf.logging.get_verbosity()
tf.logging.set_verbosity(tf.logging.ERROR) #logging 우회하기
from tensorflow.examples.tutorials.mnist import input_data

mnist = input_data.read_data_sets('MNIST_data', one_hot=True, reshape=False)

X = tf.placeholder(tf.float32, shape=[None, 28,28,1])
Y_Label = tf.placeholder(tf.float32, shape=[None,10])

C:\Anaconda\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


Extracting MNIST_data\train-images-idx3-ubyte.gz
Extracting MNIST_data\train-labels-idx1-ubyte.gz
Extracting MNIST_data\t10k-images-idx3-ubyte.gz
Extracting MNIST_data\t10k-labels-idx1-ubyte.gz


In [2]:
Kernel1 = tf.Variable(tf.truncated_normal(shape=[3,3,1,3], stddev=0.1))
Bias1 = tf.Variable(tf.truncated_normal(shape=[3], stddev=0.1))
Conv1 = tf.nn.conv2d(X, Kernel1, strides=[1,1,1,1], padding='SAME') + Bias1
Activation1 = tf.nn.sigmoid(Conv1)
Pool1 = tf.nn.max_pool(Activation1, ksize=[1,2,2,1], strides=[1,2,2,1], padding='SAME')

In [3]:
Kernel2 = tf.Variable(tf.truncated_normal(shape=[3,3,3,6], stddev=0.1))
Bias2 = tf.Variable(tf.truncated_normal(shape=[6], stddev=0.1))
Conv2 = tf.nn.conv2d(Pool1, Kernel2, strides=[1,1,1,1], padding='SAME') + Bias2
Activation2 = tf.nn.sigmoid(Conv2)
Pool2 = tf.nn.max_pool(Activation2, ksize=[1,2,2,1], strides=[1,2,2,1], padding='SAME')

In [4]:
Kernel3 = tf.Variable(tf.truncated_normal(shape=[3,3,6,12], stddev=0.1))
Bias3 = tf.Variable(tf.truncated_normal(shape=[12], stddev=0.1))
Conv3 = tf.nn.conv2d(Pool2, Kernel3, strides=[1,1,1,1], padding='SAME') + Bias3
Activation3 = tf.nn.sigmoid(Conv3)
Pool3 = tf.nn.max_pool(Activation3, ksize=[1,2,2,1], strides=[1,2,2,1], padding='SAME')

In [5]:
Kernel4 = tf.Variable(tf.truncated_normal(shape=[3,3,12,24], stddev=0.1))
Bias4 = tf.Variable(tf.truncated_normal(shape=[24], stddev=0.1))
Conv4 = tf.nn.conv2d(Pool3, Kernel4, strides=[1,1,1,1], padding='SAME') + Bias4
Activation4 = tf.nn.sigmoid(Conv4)
Pool4 = tf.nn.max_pool(Activation4, ksize=[1,1,1,1], strides=[1,1,1,1], padding='SAME')

In [6]:
W1 = tf.Variable(tf.truncated_normal(shape=[24 * 4 * 4, 10]))
B1 = tf.Variable(tf.truncated_normal(shape=[10]))
Pool4_flat = tf.reshape(Pool4, [-1, 24 * 4 * 4])
OutputLayer = tf.matmul(Pool4_flat, W1) + B1

In [7]:
with tf.name_scope("loss"):
    Loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(
        labels=Y_Label, logits=OutputLayer))
    train_step = tf.train.RMSPropOptimizer(0.005).minimize(Loss)
    tf.summary.scalar('loss', Loss)
    
with tf.name_scope("accuracy"):
    correct_predicition = tf.equal(tf.argmax(OutputLayer, 1), tf.argmax(Y_Label, 1))
    accuracy = tf.reduce_mean(tf.cast(correct_predicition, tf.float32))
    tf.summary.scalar('accuracy', accuracy)

import timeit
timeS = timeit.default_timer()

with tf.Session() as sess:
    print("Start.....")
    writer = tf.summary.FileWriter("./logs/on_logs", sess.graph)
    merged = tf.summary.merge_all()
    
    sess.run(tf.global_variables_initializer())
    summary, acc = sess.run([merged, accuracy], feed_dict={X: mnist.test.images, Y_Label: mnist.test.labels})
    for i in range(150):
        trainingData, Y = mnist.train.next_batch(64)
        sess.run(train_step, feed_dict={X: trainingData, Y_Label: Y})
        if i % 10 == 0:
            print('정확도 : ',end="")
            print(sess.run(accuracy,
                           feed_dict={X:mnist.test.images,Y_Label:mnist.test.labels}))
        summary, acc = sess.run([merged, accuracy],
                               feed_dict={X:mnist.test.images,Y_Label:mnist.test.labels})
        writer.add_summary(summary, i)

timeE = timeit.default_timer()
print("훈련시간 : ", timeE-timeS)

Start.....
정확도 : 0.0892
정확도 : 0.101
정확도 : 0.1135
정확도 : 0.1135
정확도 : 0.095
정확도 : 0.1028
정확도 : 0.0892
정확도 : 0.0974
정확도 : 0.101
정확도 : 0.1028
정확도 : 0.0982
정확도 : 0.0974
정확도 : 0.0958
정확도 : 0.1028
정확도 : 0.1135
훈련시간 :  326.7074934
